\title{myHDL Two Word Complex Multiplier}
\author{Steven K Armour}
\maketitle

This notebook/Program is a walkthrough on how to design and construct a two number complex Multiplier unit in myHDL based on the example of _Guenter Dannoritzer_

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Refrances" data-toc-modified-id="Refrances-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Refrances</a></span></li><li><span><a href="#Nonstandard-Libraries-and-tools-utilized" data-toc-modified-id="Nonstandard-Libraries-and-tools-utilized-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Nonstandard Libraries and tools utilized</a></span></li><li><span><a href="#Complex-Multiplication-Review" data-toc-modified-id="Complex-Multiplication-Review-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Complex Multiplication Review</a></span></li><li><span><a href="#Encoding-Real-and-Complex-in-a-Single-Input" data-toc-modified-id="Encoding-Real-and-Complex-in-a-Single-Input-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Encoding Real and Complex in a Single Input</a></span><ul class="toc-item"><li><span><a href="#Issue:" data-toc-modified-id="Issue:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Issue:</a></span></li><li><span><a href="#!-Negative-number-Issue-in-MyHDL-with-concat" data-toc-modified-id="!-Negative-number-Issue-in-MyHDL-with-concat-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>! Negative number Issue in MyHDL with <code>concat</code></a></span></li></ul></li><li><span><a href="#Algorithm-test" data-toc-modified-id="Algorithm-test-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Algorithm test</a></span><ul class="toc-item"><li><span><a href="#-" data-toc-modified-id="--5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span> </a></span></li><li><span><a href="#-" data-toc-modified-id="--5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span> </a></span></li><li><span><a href="#-" data-toc-modified-id="--5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span> </a></span></li><li><span><a href="#-" data-toc-modified-id="--5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span> </a></span></li><li><span><a href="#-" data-toc-modified-id="--5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span> </a></span></li></ul></li><li><span><a href="#myHDL-Implementation" data-toc-modified-id="myHDL-Implementation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>myHDL Implementation</a></span><ul class="toc-item"><li><span><a href="#Concat-issue" data-toc-modified-id="Concat-issue-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Concat issue</a></span></li></ul></li><li><span><a href="#Testing" data-toc-modified-id="Testing-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Conversion-from-myHDL-to-verilog/vhdl" data-toc-modified-id="Conversion-from-myHDL-to-verilog/vhdl-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Conversion from myHDL to verilog/vhdl</a></span><ul class="toc-item"><li><span><a href="#!!-Concat-issue-in-synthesis" data-toc-modified-id="!!-Concat-issue-in-synthesis-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>!! Concat issue in synthesis</a></span></li></ul></li></ul></div>

# Refrances
Guenter Dannoritzer, Complex Math, http://old.myhdl.org/doku.php/projects:cplx_math [3-29-18]

# Nonstandard Libraries and tools utilized 
[myHDL](http://www.myhdl.org/), [myhdlpeek by xesscorp] (using the experimental pandas branch)(https://github.com/xesscorp/myhdlpeek), [draw.io](https://www.draw.io/) 

In [1]:
from myhdl import *
from myhdlpeek import Peeker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sympy import *
init_printing()

import random

# Complex Multiplication Review

In [2]:
A, B=symbols('A, B')
a, b, c, d=symbols('a, b, c, d', real=True)

In [3]:
Multi=A*B; Multi

In [4]:
simplify(Multi.subs({A:1+1j, B:1+1j}))

In [5]:
Multi=Multi.subs({A:(a+1j*b), B:(c+1j*d)})
Multi

In [6]:
Multi=expand(Multi); Multi

In [7]:
re(Multi), im(Multi)

# Encoding Real and Complex in a Single Input

To encode the real number word and imaginary word into a single word we need to not treat the signal as a number but as a word and then contract the word into a signal word based on a well-posed contraction rule. 

The contraction rule for creating a multiplication is that both the real and imaginary part of the whole number is a 2's complement number(word) that we then contract via the first word as $Re$ part and the second word is the $Im$ part such that we then have

$$Re(\text{word})+Im(\text{word})j\Rightarrow Re(\text{word})Im(\text{word})=\mathbb{Z}(\text{word} ) $$

## Issue:
There is an ongoing issue with the `concat` function in `myHDL` that is ongoing [Issue Reopened 3-29-18] https://github.com/myhdl/myhdl/issues/128#issuecomment-377370353
When that issue is resolved this notebook will be updated accordingly. But for the moment, an attempt to develop this algorithm is shown and the problem with `concat` is highlighted so this notebook/program can be used as a benchmark for getting `concat` working properly. 

In [8]:
BitWidth=32
WordSize=BitWidth

In [9]:
ReWordLen=WordSize//2; ImWordLen=WordSize//2
print(f'The Re WordLen is {ReWordLen} bits and the Im WordLen is {ImWordLen} bits')

The Re WordLen is 16 bits and the Im WordLen is 16 bits


In [10]:
ReMax=(2**(ReWordLen-1)-1); ReMin=-2**(ReWordLen-1)
ImMax=(2**(ImWordLen-1)-1); ImMin=-2**(ImWordLen-1)
ReMax, ReMin, ImMax, ImMin

Test Tragets

In [11]:
R=(2**ReWordLen)//4; I=(2**ImWordLen)//8
print(f'Re: {R}, Im: {I}')

Re: 16384, Im: 8192


In [12]:
RN=intbv(R, min=ReMin, max=ReMax); RN
RNBin=''.join([str(int(i)) for i in RN])
bin(R, ReWordLen), RNBin, bin(R, ReWordLen)==RNBin

('0100000000000000', '0100000000000000', True)

In [13]:
IN=intbv(I, min=ImMin, max=ImMax); IN
INBin=''.join([str(int(i)) for i in IN])
bin(I, ImWordLen), INBin, bin(I, ImWordLen)==INBin

('0010000000000000', '0010000000000000', True)

In [14]:
AN=concat(RN, IN)
ANBin=''.join([str(int(i)) for i in AN])
bin(AN, WordSize), ANBin, bin(AN, WordSize)==ANBin 

('01000000000000000010000000000000', '01000000000000000010000000000000', True)

So now we can then Split `AN` to get  back `RN` and `IN`

In [15]:
RNBack=AN[:ReWordLen]; INBack=AN[ImWordLen:]
RNBack, RN, RNBack==RN,INBack, IN, INBack==IN

(intbv(16384), intbv(16384), True, intbv(8192), intbv(8192), True)

But since myHDL implements 2's complement, we need to test for negative numbers where the leading bit is the signed signal

In [16]:
TestNegNum=-26
print(f"""Target: {TestNegNum}
Absolote Bin: {bin(abs(TestNegNum), 8)}, 
Signed Bin: {bin(TestNegNum, 8)}""")

Target: -26
Absolote Bin: 00011010, 
Signed Bin: 11100110


In [17]:
TestNegNumBV=intbv(TestNegNum)[8:]
TestNegNumBV, TestNegNumBV.signed()

(intbv(230), intbv(-26))

In [18]:
R=-R; I=-I
print(f'Re: {R}, Im: {I}')

Re: -16384, Im: -8192


In [19]:
RN=intbv(R, min=ReMin, max=ReMax); RN
RNBin=''.join([str(int(i)) for i in RN])
RN.signed(), bin(R, ReWordLen), RNBin, bin(R, ReWordLen)==RNBin

(intbv(-16384), '1100000000000000', '1100000000000000', True)

In [20]:
IN=intbv(I, min=ImMin, max=ImMax); IN
INBin=''.join([str(int(i)) for i in IN])
IN.signed(), bin(I, ImWordLen), INBin, bin(I, ImWordLen)==INBin

(intbv(-8192), '1110000000000000', '1110000000000000', True)

## ! Negative number Issue in MyHDL with `concat`
At the moment `concat` does not handle a negative number as its leading term. Due to it setting the returned `intbv` min to 0

In [21]:
AN=concat(RN, IN).signed()
ANBin=''.join([str(int(i)) for i in AN])
bin(AN, WordSize), ANBin, bin(AN, WordSize)==ANBin 

AN

ValueError: intbv value -1073684480 < minimum 0

In [22]:
RNBack=AN[:ReWordLen]; INBack=AN[ImWordLen:]
RNBack.signed(), RN.signed(), RNBack.signed()==RN.signed(), INBack.signed(), IN.signed(), INBack.signed()==IN.signed()

(intbv(16384), intbv(-16384), False, intbv(8192), intbv(-8192), False)

# Algorithm test

Here we prototype the algorithm using myHDL types without explicit HDL type code that will be developed in the final algorithm.

##  
Calc the min/max for the real and imaginary number in 2's complement based on the allowed word size for real and imaginary

In [23]:
ReMax=(2**(ReWordLen-1)-1); ReMin=-2**(ReWordLen-1)
ReMax, ReMin

In [24]:
ImMax=(2**(ImWordLen-1)-1); ImMin=-2**(ImWordLen-1)
ImMax, ImMin

Create a Test A number

In [25]:
AVal=43-78j; AVal

(43-78j)

##  
Create the $a$ and $b$ part of $A$, concat $a$ and $b$, Confirm that separating $A$ yields the original $a$ and $b$

In [26]:
a=intbv(int(np.real(AVal)), min=ReMin, max=ReMax); a.signed()

intbv(43)

In [27]:
b=intbv(int(np.imag(AVal)), min=ImMin, max=ImMax); b.signed()

intbv(-78)

In [28]:
A=concat(a, b); A, A.signed()

(intbv(2883506), intbv(2883506))

In [29]:
a=intbv(A[:ReWordLen].signed(), min=ReMin, max=ReMax)
b=intbv(A[ImWordLen:].signed(), min=ImMin, max=ReMax)
a, b, 

(intbv(43), intbv(-78))

 Perform the same action as above but on $B$

In [30]:
BVal=1+123j; BVal

(1+123j)

In [31]:
c=intbv(int(np.real(BVal)), min=ReMin, max=ReMax); c

intbv(1)

In [32]:
d=intbv(int(np.imag(BVal)), min=ImMin, max=ImMax); d

intbv(123)

In [33]:
B=concat(c, d); B, B.signed()

(intbv(65659), intbv(65659))

In [34]:
c=intbv(B[:ReWordLen].signed(), min=ReMin, max=ReMax)
d=intbv(B[ImWordLen:].signed(), min=ImMin, max=ImMax)
c, d

(intbv(1), intbv(123))

##  
Use the separated $a, b, c, d$ from $A, B$ to then find the sup-product of $AB=(ac−bd)+i(ad+bc)$
$$ac, bd \in Re$$ $$ ad, bc \in Im$$

In [35]:
ac=intbv(a*c, min=ReMin, max=ReMax)
np.real(AVal)*np.real(BVal), ac, ac.signed()

(43.0, intbv(43), intbv(43))

In [36]:
bd=intbv(b.signed()*d.signed(), min=ImMin, max=ImMax)
np.imag(AVal)*np.imag(BVal), bd, bd.signed()

(-9594.0, intbv(-9594), intbv(-9594))

In [37]:
ad=intbv(a.signed()*d.signed(), min=min(ReMin, ImMin), max=max(ReMax, ImMax))
np.real(AVal)*np.imag(BVal), ad, ad.signed()

(5289.0, intbv(5289), intbv(5289))

In [38]:
bc=intbv(b.signed()*c.signed(), min=min(ReMin, ImMin), max=max(ReMax, ImMax))
np.imag(AVal)*np.real(BVal), bc, bc.signed()

(-78.0, intbv(-78), intbv(-78))

##  
Find the difference of the real and the sum of the imaginary  products 

In [39]:
Re=intbv(ac-bd, min=min(ac.min, bd.min), max=max(ac.max, bd.max))
np.real(AVal)*np.real(BVal)-np.imag(AVal)*np.imag(BVal), Re.signed()

(9637.0, intbv(9637))

In [40]:
Im=intbv(ad+bc, min=min(ad.min, bc.min), max=max(ad.max, bc.max))
np.real(AVal)*np.imag(BVal)+np.imag(AVal)*np.real(BVal), Im, Im.signed()

(5211.0, intbv(5211), intbv(5211))

In [41]:
AVal*BVal

(9637+5211j)

##  
Concat the $Re$ and $Im$ and according to the concatenation rule to a single number

In [42]:
Result=concat(Re, Im)
bin(Re, Re._nrbits)+bin(Im, Im._nrbits), ''.join([str(int(i)) for i in Result])

('00100101101001010001010001011011', '00100101101001010001010001011011')

# myHDL Implementation
## Concat issue
The 'concat' is not seeing the inputs as Signal `intbv`

<p align='center'>
<img src='ComplexMultiplierDrawings.png'>


In [43]:
@block
def CompMulti(A, B, C, ReWordLen=ReWordLen, ImWordLen=ImWordLen):
    """
    Module to implement  the complex multiplication of two complex numbers
    
    Inputs:
        A: A 2's `concat` of the input complex number where
        the input word is a ` concat` according to ReIm
        B: A 2's `concat` of the input complex number where
        input word is a `concat` according to ReIm
    Outputs:
        C: A 2's `concat` of the output  complex number product
        where output world is a `concat` according to ReIm
        
    Conversion Parameters;
        ReWordLen: The word len of the real part of `concat` word
        ImWordLen: The word len of the imag part of `concat` word
    """ 
    #calc the min max of the based on ReWord and ImWord length
    ReMax=(2**(ReWordLen-1)-1); ReMin=-2**(ReWordLen-1)
    ImMax=(2**(ImWordLen-1)-1); ImMin=-2**(ImWordLen-1)
    
    #create the regeistors to hold the procuct results
    ac=Signal(intbv(0, min=ReMin, max=ReMax))
    bd=Signal(intbv(0, min=ImMin, max=ImMax))
    ad=Signal(intbv(0, min=min(ReMin, ImMin), max=max(ReMax, ImMax)))
    bc=Signal(intbv(0, min=min(ReMin, ImMin), max=max(ReMax, ImMax)))
    
    @always_comb
    def SepMulti():
        """
        The combinational logic to separate input words into components and 
        find multiplication products
        """
        ac.next=A[:ReWordLen] *B[:ReWordLen]
        bd.next=A[ImWordLen:]*B[ImWordLen:]
        ad.next=A[:ReWordLen]*B[ImWordLen:]
        bc.next=A[ImWordLen:]*B[:ReWordLen]
    
    #will fix when Concat is working properly
    #@always_comb
    #def AddSupConcat():
    #    """
    #    Perfrom the real sup, im addtion, and concat based on
    #    ReIm to final output
    #    """
    #    
    #    C.next=concat(ac-bd, ad+bc)
        
    
    return instances()






# Testing

Generate random number within word size bounds for testing

In [44]:
#calc Bounds
Max=(2**(WordSize-1)-1); Min=-2**(WordSize-1)
ReMax=(2**(ReWordLen-1)-1); ReMin=-2**(ReWordLen-1)
ImMax=(2**(ImWordLen-1)-1); ImMin=-2**(ImWordLen-1)

#Create Testing Data
Refrance=pd.DataFrame(columns=['a', 'b', 'c', 'd'])
#Generate Testing data and bind values to DF
for i in range(20):
    a=random.sample(range(0, ReMax), 1)[0]
    b=random.sample(range(ImMin, ImMax), 1)[0]
    c=random.sample(range(0, ReMax), 1)[0]
    d=random.sample(range(ImMin, ImMax), 1)[0]
    Refrance.loc[Refrance.shape[0]]=[a, b, c, d]
    
#force the stored value in DF to be ints    
Refrance=Refrance.astype(int)
Refrance

,a,b,c,d
0,3143,-15296,13469,30206
1,10215,14709,21412,17320
2,15744,-12580,26497,-1141
3,30468,4266,6744,21633
4,23561,-25434,15982,-2695
5,19639,-32670,27638,18119
6,6150,4450,27930,30019
7,25268,14297,6706,-28529
8,18273,22261,20847,-25957
9,20020,7935,28408,-6459


In [45]:
#calc exspected results
RefranceResults=Refrance.copy()
RefranceResults['A']=RefranceResults['a']+1j*RefranceResults['b']
RefranceResults['B']=RefranceResults['c']+1j*RefranceResults['d']
RefranceResults['C']=RefranceResults['A']*RefranceResults['B']
RefranceResults

,a,b,c,d,A,B,C
0,3143,-15296,13469,30206,(3143-15296j),(13469+30206j),(504364043-111084366j)
1,10215,14709,21412,17320,(10215+14709j),(21412+17320j),(-36036300+491872908j)
2,15744,-12580,26497,-1141,(15744-12580j),(26497-1141j),(402814988-351296164j)
3,30468,4266,6744,21633,(30468+4266j),(6744+21633j),(113189814+687884148j)
4,23561,-25434,15982,-2695,(23561-25434j),(15982-2695j),(308007272-469983083j)
5,19639,-32670,27638,18119,(19639-32670j),(27638+18119j),(1134730412-547094419j)
6,6150,4450,27930,30019,(6150+4450j),(27930+30019j),(38184950+308905350j)
7,25268,14297,6706,-28529,(25268+14297j),(6706-28529j),(577326321-624995090j)
8,18273,22261,20847,-25957,(18273+22261j),(20847-25957j),(958766008-10237194j)
9,20020,7935,28408,-6459,(20020+7935j),(28408-6459j),(619980325+96108300j)


In [46]:
Peeker.clear()
Max=(2**(WordSize-1)-1); Min=-2**(WordSize-1)
ReMax=(2**(ReWordLen-1)-1); ReMin=-2**(ReWordLen-1)
ImMax=(2**(ImWordLen-1)-1); ImMin=-2**(ImWordLen-1)

A=Signal(intbv(0, min=Min, max=Max)); Peeker(A, 'A')
B=Signal(intbv(0, min=Min, max=Max)); Peeker(B, 'B')
C=Signal(intbv(0, min=Min, max=Max)); Peeker(C, 'C')
a=Signal(intbv(0, min=ReMin, max=ReMax)); Peeker(a, 'a')
b=Signal(intbv(0, min=ImMin, max=ImMax)); Peeker(b, 'b')
c=Signal(intbv(0, min=ReMin, max=ReMax)); Peeker(c, 'c')
d=Signal(intbv(0, min=ImMin, max=ImMax)); Peeker(d, 'd')

DUT=CompMulti(A, B, C, ReWordLen=ReWordLen, ImWordLen=ImWordLen)

def CompMulti_TB():
    @instance
    def Test():
        for _, j in Refrance.iterrows():
            a.next, b.next, c.next, d.next =j
            print(a, b, c, d)
            #A.next=concat(a, b); B.next=concat(c, d)
            yield delay(1)
        raise StopSimulation

        
    return instances()

In [47]:
sim = Simulation(DUT, CompMulti_TB(), *Peeker.instances()).run()
Peeker.to_wavedrom()

0000 0000 0000 0000
0c47 c440 349d 75fe
27e7 3975 53a4 43a8
3d80 cedc 6781 fb8b
7704 10aa 1a58 5481
5c09 9ca6 3e6e f579
4cb7 8062 6bf6 46c7
1806 1162 6d1a 7543
62b4 37d9 1a32 908f
4761 56f5 516f 9a9b
4e34 1eff 6ef8 e6c5
3dba 7bba 7508 3c25
118e 0933 59fd 2b3d
2f0a cb43 5ce3 237d
3e89 8c8d 58b3 aa64
336a 5659 4202 b679
0584 630a 4b3e b869
2a77 d5c4 0b7f 6449
2bcf 742f 3a10 e8d5
7519 e57e 51a4 4af9


# Conversion from myHDL to verilog/vhdl 

In [48]:
@block
def CompMulti(A, B, C, ReWordLen=ReWordLen, ImWordLen=ImWordLen):
    """
    Module to implent  the complex multibtion of two conplex numbers
    
    Inputs:
        A: A 2's conplimint concat of the input complex number where
        input workd is a concat according to ReIm
        B: A 2's conplimint concat of the input complex number where
        input workd is a concat according to ReIm
    Outputs:
        C: A 2's conplimint concat of the output  complex number product
        where ouput wordd is a concat according to ReIm
        
    Conversion Parmters;
        ReWordLen: The word len of the real part of concat word
        ImWordLen: The word len of the imag part of concat word
    """ 
    #calc the min max of the based on ReWord and ImWord length
    ReMax=(2**(ReWordLen-1)-1); ReMin=-2**(ReWordLen-1)
    ImMax=(2**(ImWordLen-1)-1); ImMin=-2**(ImWordLen-1)
    
    #create the regeistors to hold the procuct results
    ac=Signal(intbv(0, min=ReMin, max=ReMax))
    bd=Signal(intbv(0, min=ImMin, max=ImMax))
    ad=Signal(intbv(0, min=min(ReMin, ImMin), max=max(ReMax, ImMax)))
    bc=Signal(intbv(0, min=min(ReMin, ImMin), max=max(ReMax, ImMax)))
    
    @always_comb
    def SepMulti():
        """
        Compintion logic to sep input words into componets and 
        find multibilication products
        """
        ac.next=A[:ReWordLen] *B[:ReWordLen]
        bd.next=A[ImWordLen:]*B[ImWordLen:]
        ad.next=A[:ReWordLen]*B[ImWordLen:]
        bc.next=A[ImWordLen:]*B[:ReWordLen]
    
    @always_comb
    def AddSupConcat():
        """
        Perfrom the real sup, im addtion, and concat based on
        ReIm to final output
        """
        
        C.next=concat(ac-bd, ad+bc)
        
    
    return instances()






In [49]:
#helper  functions to read in the .v and .vhd generated files into python
def VerilogTextReader(loc, printresult=True):
    with open(f'{loc}.v', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***Verilog modual from {loc}.v***\n\n', VerilogText)
    return VerilogText

def VHDLTextReader(loc, printresult=True):
    with open(f'{loc}.vhd', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***VHDL modual from {loc}.vhd***\n\n', VerilogText)
    return VerilogText

In [50]:
A=Signal(intbv(0, min=Min, max=Max))
B=Signal(intbv(0, min=Min, max=Max))
C=Signal(intbv(0, min=Min, max=Max))

In [51]:
DUT=CompMulti(A, B, C, ReWordLen=ReWordLen, ImWordLen=ImWordLen)

## !! Concat issue in synthesis

In [52]:
DUT.convert()

TypeError: concat: inappropriate argument type: <class 'int'>

In [53]:
_=VerilogTextReader('CompMulti', True)

***Verilog modual from CompMulti.v***

 


In [54]:
DUT.convert(hdl='VHDL')

ConversionError: in file <ipython-input-48-f950e095a50b>, line 28:
    Signal has multiple drivers: bc

In [55]:
_=VHDLTextReader('CompMulti', True)

***VHDL modual from CompMulti.vhd***

 
